<a href="https://colab.research.google.com/github/sutherlandn/ds2002-project1/blob/main/dsProject1_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Sources:

https://restcountries.com/ (country API, used for currency)

The following sources should be in my github, but here are links to where I found them:


https://github.com/geodatasource/country-borders/blob/master/GEODATASOURCE-COUNTRY-BORDERS.CSV (country border csv)

https://dev.mysql.com/doc/index-other.html (SQL world sample database, I converted it into the file called data.sqlite to make things easier before importing)


In [91]:
#importing stuff
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import os
from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.declarative import declarative_base
import requests
import sqlite3

In [92]:
#here I imported the file titled: GEODATASOURCE-COUNTRY-BORDERS.CSV
from google.colab import files
uploaded = files.upload()

Saving GEODATASOURCE-COUNTRY-BORDERS.CSV to GEODATASOURCE-COUNTRY-BORDERS.CSV


In [93]:
#here I imported the file titled: data.sqlite
from google.colab import files
uploaded = files.upload()

Saving data.sqlite to data.sqlite


In [94]:
#making a dataframe from the CSV, changing to the 2 relevant collumns, cleaning up and reading to check that it worked
try:
    border_df = pd.read_csv('/content/GEODATASOURCE-COUNTRY-BORDERS.CSV', usecols=['country_name', 'country_border_name'])
    border_df.drop_duplicates(inplace=True)
    border_df['country_border_name'].fillna('No Borders', inplace=True)
except Exception as e:
    print(f"An error occurred: {str(e)}")

#check
print(border_df.head(20))

            country_name         country_border_name
0                Andorra                      France
1                Andorra                       Spain
2   United Arab Emirates                        Oman
3   United Arab Emirates                Saudi Arabia
4            Afghanistan                       China
5            Afghanistan  Iran (Islamic Republic of)
6            Afghanistan                    Pakistan
7            Afghanistan                  Tajikistan
8            Afghanistan                Turkmenistan
9            Afghanistan                  Uzbekistan
10   Antigua and Barbuda                  No Borders
11              Anguilla                  No Borders
12               Albania                      Greece
13               Albania                  Montenegro
14               Albania             North Macedonia
15               Albania                      Serbia
16               Armenia                  Azerbaijan
17               Armenia                     G

In [114]:
#Using REST country API to get currency names for each country, printing to check that it worked
try:
    response = requests.get('https://restcountries.com/v3.1/all?fields=name,currencies')

    if response.status_code == 200:
        data = response.json()
        processed_data = []
        for country in data:
            country_name = country.get('name', {}).get('common', 'Unknown')
            currencies = country.get('currencies', {})
            currency_name = None
            if currencies:
                first_currency = list(currencies.values())[0]
                currency_name = first_currency.get('name', None)
            processed_data.append({'countryname': country_name, 'currencyname': currency_name})
        currency_df = pd.DataFrame(processed_data)
        print("Data processing successful. DataFrame created.")
    else:
        print(f"Error: Received status code {response.status_code} from the server.")
except requests.RequestException as e:
    print(f"Error: {e}")

#check
print('\n')
print(currency_df.head(10))

Data processing successful. DataFrame created.


    countryname                 currencyname
0        Cyprus                         Euro
1       Eritrea               Eritrean nakfa
2       Liberia              Liberian dollar
3       Bermuda             Bermudian dollar
4  Vatican City                         Euro
5  Cook Islands          Cook Islands dollar
6       Somalia              Somali shilling
7        Zambia               Zambian kwacha
8     Venezuela  Venezuelan bolívar soberano
9  Turkmenistan           Turkmenistan manat


In [96]:
#accessing tables from the sqlite version of the world sample database, picking the collumns I want and putting them into dataframes:
conn = sqlite3.connect('data.sqlite')
query_city = "SELECT * FROM city"
query_country = "SELECT Code, Name, Continent, Region, SurfaceArea, Population, Capital FROM country"
query_countrylanguage = "SELECT * FROM countrylanguage"

df_city = pd.read_sql_query(query_city, conn)
df_country = pd.read_sql_query(query_country, conn)
df_countrylanguage = pd.read_sql_query(query_countrylanguage, conn)

conn.close()

#checking that this worked
print(df_city.head())
print('\n')
print(df_country.head())
print('\n')
print(df_countrylanguage.head())

  ID            Name CountryCode       District Population
0  1           Kabul         AFG          Kabol    1780000
1  2        Qandahar         AFG       Qandahar     237500
2  3           Herat         AFG          Herat     186800
3  4  Mazar-e-Sharif         AFG          Balkh     127800
4  5       Amsterdam         NLD  Noord-Holland     731200


  Code         Name      Continent                     Region SurfaceArea  \
0  ABW        Aruba  North America                  Caribbean      193.00   
1  AFG  Afghanistan           Asia  Southern and Central Asia   652090.00   
2  AGO       Angola         Africa             Central Africa  1246700.00   
3  AIA     Anguilla  North America                  Caribbean       96.00   
4  ALB      Albania         Europe            Southern Europe    28748.00   

  Population Capital  
0     103000     129  
1   22720000       1  
2   12878000      56  
3       8000      62  
4    3401200      34  


  CountryCode    Language IsOfficial Perc

In [115]:
#putting all of my dataframes into a new sqlite file as tables!
conn = sqlite3.connect('world_database.db')
border_df.to_sql(name='countryneighbors', con=conn, if_exists='replace', index=False)
currency_df.to_sql(name='countrycurrency', con=conn, if_exists='replace', index=False)
df_city.to_sql(name='city', con=conn, if_exists='replace', index=False)
df_country.to_sql(name='country', con=conn, if_exists='replace', index=False)
df_countrylanguage.to_sql(name='countrylanguage', con=conn, if_exists='replace', index=False)
conn.close()

In [116]:
#now that we have this new sqlite database, let's try a query that will display: The name and continent from the country table,
#the average city population from the city table, and the total number of neighboring countries from the countryneighbor table
conn = sqlite3.connect('world_database.db')
query = """
SELECT
    co.Name AS CountryName,
    co.Continent AS CountryContinent,
    AVG(ci.Population) AS AverageCityPopulation,
    COUNT(DISTINCT cn.country_border_name) AS TotalNumberOfNeighbors
FROM
    country AS co
JOIN
    city AS ci ON co.Code = ci.CountryCode
JOIN
    countryneighbors AS cn ON co.Name = cn.country_name
GROUP BY
    co.Name
ORDER BY
    TotalNumberOfNeighbors DESC, AverageCityPopulation DESC
"""

country_language_population_data = pd.read_sql_query(query, conn)
conn.close()
print(country_language_population_data)

                 CountryName CountryContinent  AverageCityPopulation  \
0                      China             Asia          484720.699725   
1         Russian Federation           Europe          365876.719577   
2                     Brazil    South America          343507.448000   
3                    Germany           Europe          282209.494624   
4                     Turkey             Asia          456887.548387   
..                       ...              ...                    ...   
192         Christmas Island          Oceania             700.000000   
193                     Niue          Oceania             682.000000   
194  Cocos (Keeling) Islands          Oceania             335.000000   
195                  Tokelau          Oceania             300.000000   
196                 Pitcairn          Oceania              42.000000   

     TotalNumberOfNeighbors  
0                        16  
1                        14  
2                        10  
3              

And there we have it! My SQLite database has tables countryneighbors, countrycurrency, city, country, and countrylanguage, sourced from a sample SQL database, a CSV file, and a public API.